In [1]:
import os
import re
import time
from datetime import datetime

import requests

from selenium import webdriver
from selenium.webdriver.firefox.options import Options

In [2]:
import pprint

In [3]:
page_url = 'https://www.bexar.org/722/Jail-Activity-Reports'
xpath = '//a[text()="State and County"]'

In [4]:
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)
driver.get(page_url)
time.sleep(1)
link_elements = driver.find_elements_by_xpath(xpath)
links = [element.get_attribute('href') for element in link_elements]
driver.quit()

In [5]:
print(links)

['http://apps.bexar.org/jailactivity/KB50061D20191027.txt', 'http://apps.bexar.org/jailactivity/KB50061D20191028.txt', 'http://apps.bexar.org/jailactivity/KB50061D20191029.txt', 'http://apps.bexar.org/jailactivity/KB50061D20191030.txt', 'http://apps.bexar.org/jailactivity/KB50061D20191031.txt', 'http://apps.bexar.org/jailactivity/KB50061D20191101.txt', 'http://apps.bexar.org/jailactivity/KB50061D20191102.txt']


In [8]:
def get_charge_info(line):
    charge = {
        'charge': line[68:91].rstrip(),
        'action': line[92:110].rstrip(),
        'action_agent': line[110:122].rstrip(),
    }
    return charge

In [9]:
links = links[0:1]
for link in links:
    print(link)
    output_folder = os.path.expanduser('~/data/jailactivityreports')
    output_file_path = f'{output_folder}/{os.path.basename(link)}'
    report = requests.get(link).text
    line_start_pattern = re.compile('^0[A-Z]\d{9}')
    w_pattern = re.compile('W\d{7}')
    j_pattern = re.compile('J\d{7}')
    whitespace_pattern = re.compile('^\s{68}')
    date_pattern = re.compile('\d{2}/\d{2}/\d{4} \d{4}\s{43}')
    lines = report.split('\r\n')
#     print(lines[:20])
    for i, line in enumerate(lines, 1):
        if line_start_pattern.match(line):
#             print(f'{i}>>>{line}')
            print(record)
            record = {}
            charges = []
            record['booking_number'] = line[0:11]
            record['booking_date'] = datetime.strptime(line[14:29], '%m %d %Y %H%M')
            record['name'] = line[30:63].rstrip()
#             print(booking_number, booking_date, name)
        
        elif w_pattern.search(line) or j_pattern.search(line):
            try:
                record['action_date'] = datetime.strptime(line[10:25], '%m/%d/%Y %H%M')
            except:
                pass
            record['w_number'] = line[26:34].rstrip()
            record['j_number'] = line[35:43].rstrip()
            record['court'] = line[44:48]
            record['case_number'] = line[49:60].rstrip()
#             charge = line[68:91].rstrip()
#             action = line[92:110].rstrip()
#             action_agent = line[110:122].rstrip()
            charges.append(get_charge_info(line))
#             charges.append([charge, action, action_agent])
            record['charges'] = charges
#             print(action_date, w_number, j_number, court, case_number, charges)
        
        elif whitespace_pattern.match(line):
            print(">>>? Got to whitespace checking line")
#             charge = line[68:91].rstrip()
#             action = line[92:110].rstrip()
#             action_agent = line[110:122].rstrip()
#             charges.append([charge, action, action_agent])
            charges.append(get_charge_info(line))
            record['charges'] = charges
#             print(action_date, w_number, j_number, court, case_number, charges)
            
        elif date_pattern.search(line):
            record['action_date'] = datetime.strptime(line[10:25], '%m/%d/%Y %H%M')
#             charge = line[68:91].rstrip()
#             action = line[92:110].rstrip()
#             action_agent = line[110:122].rstrip()
#             charges.append([charge, action, action_agent])
            charges.append(get_charge_info(line))
            record['charges'] = charges
#             print(action_date, w_number, j_number, court, case_number, charges)
    
#         try:
#             print(booking_number, booking_date, name, action_date, w_number, j_number, court, case_number, charge, action)
#         except:
#             pass
        
#             print(f'{i}>>>{line}')
#     with open(output_file_path, 'w') as output_file:
#         output_file.write(report)

http://apps.bexar.org/jailactivity/KB50061D20191027.txt
{'booking_number': '0B201925524', 'booking_date': datetime.datetime(2019, 6, 2, 4, 54), 'name': 'PENA, JOSE', 'action_date': datetime.datetime(2019, 10, 26, 5, 21), 'w_number': 'W1687072', 'j_number': 'J1881852', 'court': 'D186', 'case_number': '2019CR3967'}
>>>? Got to whitespace checking line
{'booking_number': '0B201925524', 'booking_date': datetime.datetime(2019, 6, 2, 4, 54), 'name': 'PENA, JOSE', 'action_date': datetime.datetime(2019, 10, 26, 5, 21), 'w_number': 'W1674157', 'j_number': 'J1879819', 'court': 'CC5 ', 'case_number': '601769', 'charges': [{'charge': 'F3 ASSAULT-FAMILY-2ND O', 'action': 'CASE DISMISSED', 'action_agent': 'JD ANGELINI'}, {'charge': 'MB DRIVING WHILE INTOX-', 'action': "REL'D ON P-R BOND", 'action_agent': 'JD GONZALES'}, {'charge': 'F  PAROLE VIOLATION', 'action': 'WARRANT WITHDRAWN', 'action_agent': 'P THIELKE'}]}
{'booking_number': '0B201930182', 'booking_date': datetime.datetime(2019, 7, 2, 0, 53)